In [1]:
# Librarys
%run ForeSignalScraper.ipynb function(Login,CustomizeFoSignals)
%run TradingViewScaper.ipynb function(Price)
%run OpenOrdersMT5.ipynb function(Login_mt5,Expiration_time,Determination_Set_Order)
%run Sql-Server.ipynb function(send_datas)
from datetime import datetime
from collections import OrderedDict
import pandas as pd
import random
import time

Rows deleted successfully.


In [2]:
# Currencys list and Desired Time to Refresh wbsites
Currencys_list = ['EURUSD','USDCHF','GBPUSD','USDJPY','USDCAD','AUDUSD','EURJPY','NZDUSD','GBPCHF']
desired_time = int(input('Please Enter Your Desired Time Range To load Websiter - input a number ex:20 : '))

# Foresignal login Information
FoLogin_url = 'https://foresignal.com/en/login/index'
Fouser_name = 'xabiz'
FoUser_password = 'tohid0010011'

# Foresignal Scrape Specification
FoWebsite_url = 'https://foresignal.com/en/'
Foclassorid = 'class'
Foclassoridvalue = 'd-flex flex-row flex-wrap signal-row'

#Login to Foresignal and Scrape Data
ForeSignalDriver = Login(FoLogin_url,Fouser_name,FoUser_password)

#Login to Tradingview and Scrape Data
final_list = Price(2,Currencys_list)

# Alpari Broker Demo Account
# login_id = 51915381
# password = 'dq5tttot'
# server = 'Alpari=MT5-Demo'

# FIBOGroup Broker Demo Account
# login_id = 3060009
# password = '$9Bj#5qL'
# server = 'FIBOGroup-MT5 Server'

#MetaTrader5 Login Informations and login to Software
login_id = 51915381
password = 'dq5tttot'
server = 'Alpari-MT5-Demo'
Login_mt5 (login_id,password,server)

# Use this line to create connection with sql server database
server = 'DESKTOP-7PVA1DT'
database = 'FInancial Scraper'
username = 'sa'
password = 'Mohsen13781378###***'
conn_str = connect_to_database(server,database,username,password)

Please Enter Your Desired Time Range To load Websiter - input a number ex:20 : 60


In [3]:
# Empty Dicts and Lists
final_informations = OrderedDict()
History = OrderedDict()
FoFinalList = list()
Reference_list = list()
Order_List = list()
History_counter = 1

while True:
    
    # Get Datas From Fore Signal. If lengh data > 10 and not in FoFinalList, data appended into FoFinalList
    
    for items in CustomizeFoSignals(ForeSignalDriver,FoWebsite_url,Foclassorid,Foclassoridvalue):
        if len(items) > 10 and items not in FoFinalList:
            FoFinalList.append(items)
    
    
    # Get Current Price Every time and Update it
    
    Tradingview_price_dictionary = OrderedDict()
    for item in range (0,9):
        now = datetime.now()
        final_list[0].switch_to.window(final_list[0].window_handles[item])
        current_date = f'{Currencys_list[item]} : {now}'
        current_price = final_list[1][item].text
        Tradingview_price_dictionary[current_date] = current_price
        
    # Merge Foresignal and Current Price of Tradingvirw Datas in a Dictionary
        
    for item_fo in FoFinalList:
        for k,v in Tradingview_price_dictionary.items():
            if item_fo[0] in k[:6]:
                item_fo.append(k)
                item_fo.append(v)
                final_informations[f'{item_fo[:11]}'] = item_fo
    
    
    Reference_list_start_len = len(Reference_list)
    Reference_list = list(final_informations.items()) 
    Reference_list_final_len = len(Reference_list)
    
    
    if Reference_list_start_len < Reference_list_final_len:
        for items in Reference_list[Reference_list_start_len:Reference_list_final_len]:
            Order_List.append(items)
            
    
    for items in Order_List:

        if len(items[1])>13:
            items[1].remove(items[1][11])
            items[1].remove(items[1][11])
        
        current_time = str(datetime.now()).split()[1].replace(':',' ').split()  
        Order_hour = int(items[1][2].replace(':',' ').split()[0])    
        Order_minute = int(items[1][2].replace(':',' ').split()[1])
                    
        if int(current_time[0]) >= Order_hour:
            
            Symbole = items[1][0]
            Entry_Price = float(items[1][6])
            Takeprofit_Price = float(items[1][8])
#             Stoploss_Price = float(items[1][10])
            Current_Price = float(items[1][12])
            current_time = items[1][4].replace(':',' ').split()
            expiration = Expiration_time (int(current_time[0]),int(current_time[1]))
                    
            if items[1][5] == 'Buy at' and Entry_Price > Current_Price:
                Type_of_Trade = 'buy stop'
                Stoploss_Price = float(items[1][10])+((Entry_Price-float(items[1][10]))/2)
                
            elif items[1][5] == 'Buy at' and Entry_Price < Current_Price:
                Type_of_Trade = 'buy limit'
                Stoploss_Price = float(items[1][10])+((Entry_Price-float(items[1][10]))/2)
            
            elif items[1][5] == 'Sell at' and Entry_Price < Current_Price:
                Type_of_Trade = 'sell stop'
                Stoploss_Price = float(items[1][10])-((float(items[1][10])- Entry_Price)/2)
            
            elif items[1][5] == 'Sell at' and Entry_Price > Current_Price:
                Type_of_Trade = 'sell limit'
                Stoploss_Price = float(items[1][10])-((float(items[1][10])- Entry_Price)/2)
                        
            request = Determination_Set_Order(Type_of_Trade, Symbole , 0.05 ,Entry_Price
                                              ,Stoploss_Price,Takeprofit_Price,expiration)   
            result = mt.order_send(request)
            result
                                        
            TableName = 'Informations'
            InfoTuple = (History_counter, items[1][0], items[1][1], items[1][2], items[1][3], items[1][4],items[1][5]
                                 , float(items[1][6]), float(items[1][8]), float(items[1][10]), items[1][11][8:], float(items[1][12]))
                                 
            History_counter += 1
            send_datas(conn_str,TableName,InfoTuple)                    
            Order_List.remove(items)
            
        if len(items[1])>13:
            items[1].remove(items[1][11])
            items[1].remove(items[1][11])
        
        
    delay_time = random.randrange(0,desired_time)
    time.sleep(delay_time)


KeyboardInterrupt: 